In [2]:
import boto3
import mlflow

s3 = boto3.client("s3")
print([bucket['Name'] for bucket in s3.list_buckets()['Buckets']])


['mlflow-bucket-youtube-sentiment-project']


In [3]:
mlflow.set_tracking_uri("http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/")

In [4]:
mlflow.set_experiment("Exp 3 - TFIDF Trigram max_features")

2025/07/07 23:18:06 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TFIDF Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-youtube-sentiment-project/293823542415232213', creation_time=1751910490543, experiment_id='293823542415232213', last_update_time=1751910490543, lifecycle_stage='active', name='Exp 3 - TFIDF Trigram max_features', tags={}>

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [8]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])

In [9]:
df.shape

(36662, 2)

In [14]:
def run_experiment_TFIDF_max_features(max_features):
    ngram_range = (1, 3)
    
    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 3: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("Vectorizer_type", "TF-IDF")
        mlflow.log_param("nGram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("N_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_TFIDF_max_features(max_features)
        

2025/07/07 23:39:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/07 23:39:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213/runs/afa10e28a4c34384bfbb168c00bada34
🧪 View experiment at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213


2025/07/07 23:41:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/07 23:41:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213/runs/afaa37101d8544cb95bcd1301fa1db1e
🧪 View experiment at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213


2025/07/07 23:43:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/07 23:43:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213/runs/b48d101fc3954155b0e478a41f928cdb
🧪 View experiment at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213


2025/07/07 23:56:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/07 23:56:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213/runs/e032a92b8b0d4874af3a31725b953285
🧪 View experiment at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213


2025/07/07 23:58:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/07 23:58:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213/runs/2dcf44effb454cdaa9d07c263fb64e41
🧪 View experiment at: http://ec2-3-95-152-130.compute-1.amazonaws.com:5000/#/experiments/293823542415232213
